# Les murs...

## 1. Initialisation

In [20]:
import numpy as np
import pandas as pd
from IPython.display import display, HTML

# NOTE: array[y, x]
print("Imports: OK...")

Imports: OK...


In [21]:
# Constants
# size of arrays : LENGTHxLENGTH
WIDTH = 5
HEIGHT = 5

# if abs(a - b) < EPSILON then a == b
EPSILON = 0.1
# For approximation
ROUNDING_POSITION = 3

# Constants
# COLOR_RANGE+1 shades of grey
COLOR_RANGE = 4
COLOR_MIN = 40
COLOR_MAX = 200
COLOR_DELTA = int((COLOR_MAX - COLOR_MIN) / COLOR_RANGE)

# x axis
vx = np.array([1., 0.])

light_angles = np.array([-180., 180.])

# Display Array, descending y axis
def dumpdata(msg, data):
    print(f"## {msg} ##\n {pd.DataFrame(data).sort_index(ascending=False, axis=0)}\n")

print("Constants and basic functions: OK...")

Constants and basic functions: OK...


In [22]:
# base angles
base_alpha = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_min = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_max = np.zeros([HEIGHT, WIDTH])*np.nan

# real angles
real_alpha_min = np.zeros([HEIGHT, WIDTH])*np.nan
real_alpha_max = np.zeros([HEIGHT, WIDTH])*np.nan
real_alpha = np.zeros([HEIGHT, WIDTH])

# [0. ; 1.] ratio = entering light angle / base light angle
# are we receiving all the lighting that we can accept without blocking element?
real_ratio = np.zeros([HEIGHT, WIDTH])

# Blocking blocks
blocks = np.zeros([HEIGHT, WIDTH])

# vibility to color convertion
lighting_color = np.zeros([HEIGHT, WIDTH])

# Light Source L (0.5, 0.5)
L = np.array([0.5, 0.5])

print("Structures: OK")

Structures: OK


In [23]:
def setBaseAlpha(x, y, a, b):
    alpha = np.degrees(np.math.atan2(np.linalg.det([a,b]),np.dot(a,b)))
    beta = np.degrees(np.math.atan2(np.linalg.det([vx, a]), np.dot(vx, a)))
    #print(f"({x}, {y}) alpha:{alpha}, beta:{beta}")
    base_alpha[y, x] = alpha
    base_alpha_min[y, x] = beta
    base_alpha_max[y, x] = beta + alpha

def computeBaseSector():
    # First line
    y = 0
    for x in range(1,WIDTH):
        a = np.array([x, 0]) - L
        b = np.array([x, 1]) - L
        setBaseAlpha(x, y, a, b)
    
    # First Column
    x = 0
    for y in range(1, HEIGHT):
        a = np.array([1, y]) - L
        b = np.array([0, y]) - L
        setBaseAlpha(x, y, a, b)

    # All the other cells
    for y in range(1, HEIGHT):
        for x in range(1, WIDTH):
            a = np.array([x+1, y]) - L
            b = np.array([x, y+1]) - L
            setBaseAlpha(x, y, a, b)

computeBaseSector()
print("Base sectors: OK")

Base sectors: OK


In [24]:
def resetArrays():
    real_alpha_min.fill(np.nan)
    real_alpha_max.fill(np.nan)
    real_alpha.fill(0.)
    real_ratio.fill(0.)
    real_alpha_min[0, 0] = light_angles[0]
    real_alpha_max[0, 0] = light_angles[1]
    real_alpha[0, 0] = light_angles[1] - light_angles[0]
    real_ratio[0, 0] = 1.
    blocks.fill(0.)

# Get the intersection of 2 angles
def intersect(in_min, in_max, th_min, th_max):
    if (in_max < th_min or in_min > th_max):
        # No overlap
        return (0., 0.)
    r_max = min(th_max, in_max)
    r_min = max(th_min, in_min)
    if (r_max - r_min < EPSILON):
        r_max = r_min
    return (r_min, r_max)

# Get the union of 2 angles IF they touch each other
def union(a1_min, a1_max, a2_min, a2_max):
    # One angle is null
    if (a1_min == a1_max == 0):
        return(a2_min, a2_max)
    if (a2_min == a2_max == 0):
        return(a1_min, a1_max)
    # Not neighbors ?
    if (abs(a2_min - a1_max) > EPSILON and abs(a1_min - a2_max) > EPSILON):
        #print(a1_min, a1_max, a2_min, a2_max, abs(a2_min - a1_max))
        print(f"[union] - FATAL - ({x}, {y}) ({a1_min}, {a1_max}) not neighbor of ({a2_min}, {a2_max})")
        return (0., 0.)
    # Union
    return (min(a1_min, a2_min), max(a1_max, a2_max))

    
def setRealAlpha(x, y, a_min, a_max):
    real_alpha_min[y, x] = a_min
    real_alpha_max[y, x] = a_max
    real_alpha[y, x] = a_max - a_min
    # Watch out for the 1.0 -> 0.9999... 1.0 effect
    real_ratio[y, x] = round(real_alpha[y, x] / base_alpha[y, x], ROUNDING_POSITION)
    # if we block lighting keep only the ratio and reset all other elements
    # with the ratio we know if the blocking cell receive light (obstacle must be displayed)
    if (blocks[y, x] == 1):
        #real_alpha[y, x] = 0.
        real_alpha_min[y, x] = 0.
        real_alpha_max[y, x] = 0.
    return(real_alpha_min[y, x], real_alpha_max[y, x])

# For each cell, compute the incident light
def computeSector(x, y):
    # Already done
    if (not np.isnan(real_alpha_min[y, x]) and not np.isnan(real_alpha_max[y, x])):
        return (real_alpha_min[y, x], real_alpha_max[y, x])
    # Origin
    if (x == 0 and y == 0):
        return (real_alpha_min[0, 0], real_alpha_max[0, 0])
    # First row (only x-1 is needed)
    if (y == 0):
        prevx = computeSector(x-1, y)
        real = intersect(prevx[0], prevx[1], base_alpha_min[y, x], base_alpha_max[y, x])
        return setRealAlpha(x, y, real[0], real[1])
    # First column (only y-1 is needed)
    if (x == 0):
        prevy = computeSector(x, y-1)
        real = intersect(prevy[0], prevy[1], base_alpha_min[y, x], base_alpha_max[y, x])
        return setRealAlpha(x, y, real[0], real[1])
    # The other cells (x-1 AND y-1 control the flow of light entering this cell)
    else:
        prevx = computeSector(x-1, y)
        prevy = computeSector(x, y-1)
        real = intersect(prevx[0], prevx[1], base_alpha_min[y, x], base_alpha_max[y, x])
        real2 = intersect(prevy[0], prevy[1], base_alpha_min[y, x], base_alpha_max[y, x])
        real = union(real[0], real[1], real2[0], real2[1])
        return setRealAlpha(x, y, real[0], real[1])

def convertLightingRatioToColor():
    for y in range(HEIGHT):
        for x in range(WIDTH):
            if (real_ratio[y, x] == 0.):
                lighting_color[y, x] = 0
            else:
                lighting_color[y, x] = int(abs(real_ratio[y, x]) * COLOR_RANGE) * COLOR_DELTA + COLOR_MIN

def color_max_red(value):
    v = int(value)
    hexcolor = '#%02x%02x%02x' % (v, v, v)
    return 'background-color: {}'.format(hexcolor)

def color_block(value):
    return 'font-weight: bold; color: red'

def markBlocks(datastyle):
    for y in range(HEIGHT):
        for x in range(WIDTH):
            if (blocks[y, x]):
                datastyle.applymap(color_block, subset=(y, x))

def fakeDisplay(msg=""):
    df = pd.DataFrame(lighting_color).sort_index(ascending=False, axis=0)
    s = df.style.applymap(color_max_red)
    markBlocks(s)
    if msg!="":
        display(HTML("<b>"+msg+"</b>"))
    display(HTML(s.render()+"</br>"))

print("Functions: OK")      

Functions: OK


## 2. Enchainement de murs

- où on observe comment le ratio est modifié par la présence de murs attenants...

In [25]:
resetArrays()

blocks[2, 1] = 1
blocks[2, 2] = 1

computeSector(WIDTH-1, HEIGHT-1)
convertLightingRatioToColor()

print(f"Light source angles ({light_angles[0]}, {light_angles[1]})\n")

dumpdata("Real Ratio", real_ratio)
fakeDisplay()


Light source angles (-180.0, 180.0)

## Real Ratio ##
      0      1    2      3      4
4  1.0  0.295  0.0  0.000  0.000
3  1.0  0.139  0.0  0.000  0.120
2  1.0  1.000  0.5  0.356  0.733
1  1.0  1.000  1.0  1.000  1.000
0  1.0  1.000  1.0  1.000  1.000



,0,1,2,3,4
4,200,80,0,0,0
3,200,40,0,0,40
2,200,200,120,80,120
1,200,200,200,200,200
0,200,200,200,200,200


#### Constatations

- Pour le calcul du ratio réel du mur en (2, 2) on a besoin de :
    - le secteur angulaire traversant la cellule C(2,2) va du bord haut gauche au bord bas droite
    - les voisins contribuant à la lumière reçu par C(2,2) sont C(1, 2) et C(2, 1)
- son voisin de gauche C(1,2) étant un mur, il bloque la transmission de la lumière
- C(2,2) reçoit donc un éclairage partiel (reçu / base).
- même phénomène avec un enchaînement vertical

In [26]:
resetArrays()

blocks[1, 2] = 1
blocks[2, 2] = 1

computeSector(WIDTH-1, HEIGHT-1)
convertLightingRatioToColor()

print(f"Light source angles ({light_angles[0]}, {light_angles[1]})\n")
dumpdata("Real Ratio", real_ratio)    
fakeDisplay()

Light source angles (-180.0, 180.0)

## Real Ratio ##
      0    1      2      3      4
4  1.0  1.0  0.733  0.120  0.000
3  1.0  1.0  0.356  0.000  0.000
2  1.0  1.0  0.500  0.000  0.000
1  1.0  1.0  1.000  0.139  0.295
0  1.0  1.0  1.000  1.000  1.000



,0,1,2,3,4
4,200,200,120,40,0
3,200,200,80,0,0
2,200,200,120,0,0
1,200,200,200,40,80
0,200,200,200,200,200


#### précisions ...

- le ratio dont il est question sert à savoir si la vision qu'on a de la cellule cible à partir de la cellule C(0,0) est bloqué ou non (et à quel point)
- **ce ratio n'est pas lié à l'éclairage réel** (en particulier avec la luminance)

#### ... et interrogation(s)

[R1] vu toutes les approximations qui sont faites dans le jeu (une seule valeur pour représenter la quantité de lumière perçue en provenance d'une cellule complète), doit on travailler calculer plusieurs valeurs pour modéliser la présence d'obstacle sur le chemin allant de la cellule C(0,0) à une autre cellule C(x,y) ET la quantité de lumière perçue par un observateur en C(0,0) ?

Exemple : soit trois cellules A (0,3), B (1,3), C (2,3) occupées par des murs et L la source lumineuse située en (0.5, 0.5) avec une couverture de 360°. On considère que la surface d'une face d'une cellule fait 1m^2, que les murs se comportent en diffuseur parfait

- l'intensité lumineuse varie en fonction du carré de la distance
- d^2 vaut 6.25 pour A, 7.2361 pour B et 10.24 pour C donc de A à B on a un facteur de 0.864, de A à C 0.610

- l'éclairement du mur E(x,y) = I * cos(alpha) / d^2 (I l'intensité lumineuse de la source L en candela ; alpha l'angle de la direction principale d'émission et la normale de la surface éclairée) 

- Pour avoir une idée, on considère que I = 1000 candela
    - E(A) = 1000*1/6.25 = 1600 ; E(B) = cos(21.8)/7.2361 = 1283.13 ; E(C) = cos(38.66)/10.24 = 762.56

- Comme le mur est un diffuseur parfait, on applique la loi de Lambert : L = ρE/π (avec ρ le facteur de réflexion diffuse de la surface ). Du coup, ça revient à L=kE Ce qui ne change pas le rapport entre E(A), E(B) et E(C).


La luminance est à peu près 2 fois moins forte en provenance de C que de A.

Bon, mais quelle différence entre le rapport du ratio "couverture" (le real_ratio utilisé jusque là) de deux cellules du mur et le rapport entre les valeurs de luminance de ces mêmes cellules ?

Faisons le test avec un mur horizontal de 100 cellules.

- les 2 ratios calculés sont pour les cellules C et A du mur avec C une cellule quelconque et A la cellule pour laquelle la normale du mur et la direction principale de la lumière on la même direction (x=0, y=3 dans ce cas)
    - (couverture réelle C / couverture théorique C) / couverture réelle A / couverture théorique A)
    - Eclairement E de C / Eclairement E de A

In [27]:
# Constants
# size of arrays : LENGTHxLENGTH
WIDTH = 50
HEIGHT = 4

# Compute base angles
base_alpha = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_min = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_max = np.zeros([HEIGHT, WIDTH])*np.nan

lighting_color = np.zeros([HEIGHT, WIDTH])

real_alpha_min = np.zeros([HEIGHT, WIDTH])*np.nan
real_alpha_max = np.zeros([HEIGHT, WIDTH])*np.nan
real_alpha = np.zeros([HEIGHT, WIDTH])

# Blocking blocks
blocks = np.zeros([HEIGHT, WIDTH])

# [0. ; 1.] ratio = entering light angle / base light angle
# are we receiving all the lighting that we can accept without blocking element?
real_ratio = np.zeros([HEIGHT, WIDTH])

resetArrays()

for x in range(WIDTH):
    blocks[HEIGHT-1, x] = 1

computeBaseSector()
computeSector(WIDTH-1, HEIGHT-1)
convertLightingRatioToColor()

print(f"Light source angles ({light_angles[0]}, {light_angles[1]})\n")


# Version luminance
y=HEIGHT-1
vy = -1.*np.array([0., -1.])
ref_E = 0.

lum_ratio=np.zeros([WIDTH])
ratio_diff=np.zeros([WIDTH])
print("pour chaque x, affiche les 2 ratios et le l'erreur")
I = 10000
for x in range(0, WIDTH):
    a = np.array([x+0.5, HEIGHT-1+0.5]) - L
    b= vy
    alpha = np.math.atan2(np.linalg.det([a,b]),np.dot(a,b))
    d = abs(np.linalg.norm([a]))
    E  = I*np.math.cos(alpha)/(d*d)
    if (x==0):
        lum_ratio[x] = 1.
        ref_E = E
    else:
        lum_ratio[x] = E / ref_E
    # abs(Th - Exp) / Th
    ratio_diff[x] = abs(lum_ratio[x] - real_ratio[HEIGHT-1, x]) / lum_ratio[x]
    print(f"({x}) (Lum ratio, Vision ratio) = ({lum_ratio[x]}, {real_ratio[HEIGHT-1, x]}) err={ratio_diff[x]}")

dumpdata("Real Ratio", real_ratio)    
fakeDisplay()

Light source angles (-180.0, 180.0)

pour chaque x, affiche les 2 ratios et le l'erreur
(0) (Lum ratio, Vision ratio) = (1.0, 1.0) err=0.0
(1) (Lum ratio, Vision ratio) = (0.8538149682454622, 0.861) err=0.008415209409250082
(2) (Lum ratio, Vision ratio) = (0.5760348191569687, 0.644) err=0.11798797326609321
(3) (Lum ratio, Vision ratio) = (0.35355339059327384, 0.5) err=0.41421356237309476
(4) (Lum ratio, Vision ratio) = (0.21600000000000003, 0.407) err=0.884259259259259
(5) (Lum ratio, Vision ratio) = (0.13619005290728647, 0.343) err=1.5185392962105881
(6) (Lum ratio, Vision ratio) = (0.0894427190999916, 0.298) err=2.331741286474686
(7) (Lum ratio, Vision ratio) = (0.061125408400215665, 0.264) err=3.3189895480365994
(8) (Lum ratio, Vision ratio) = (0.04328919142870583, 0.237) err=4.474807733249577
(9) (Lum ratio, Vision ratio) = (0.03162277660168379, 0.216) err=5.8305197459636995
(10) (Lum ratio, Vision ratio) = (0.023725972202725765, 0.198) err=7.3452850027891685
(11) (Lum ratio, Visio

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
3,200,160,120,120,80,80,80,80,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,0,0,0,0,0,0,0,0,0,0,0,0
2,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200
1,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200
0,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200


Test d'affichage en remplaçant real_ratio par celui correspondant à la luminance

In [28]:
y=HEIGHT-1
for x in range(0, WIDTH):
    real_ratio[HEIGHT-1, x] = round(lum_ratio[x], ROUNDING_POSITION)

dumpdata("Real Ratio", real_ratio)    
convertLightingRatioToColor()
fakeDisplay()

## Real Ratio ##
     0      1      2      3      4      5      6      7      8      9  ...   \
3  1.0  0.854  0.576  0.354  0.216  0.136  0.089  0.061  0.043  0.032 ...    
2  1.0  1.000  1.000  1.000  1.000  1.000  1.000  1.000  1.000  1.000 ...    
1  1.0  1.000  1.000  1.000  1.000  1.000  1.000  1.000  1.000  1.000 ...    
0  1.0  1.000  1.000  1.000  1.000  1.000  1.000  1.000  1.000  1.000 ...    

    40   41   42   43   44   45   46   47   48   49  
3  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0  
2  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
1  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  
0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  1.0  

[4 rows x 50 columns]



,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49
3,200,160,120,80,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,40,0,0,0,0,0,0,0,0,0,0,0,0
2,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200
1,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200
0,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200,200


#### Moralité de l'histoire

Bref, les deux calculs ne conduisent pas du tout aux mêmes résultats. **Faut-il distinguer la question de couverture en fonction des obstacles de celle d'éclairage ?** (pour quel gain et quels coût)

### Supposition : juste la couverture

- on laisse tomber la question de l'éclairage (sera l'object d'un autre algo ou on s'en moque complètement)
- on commence par remettre les tableaux en état

In [29]:
WIDTH = 5
HEIGHT = 5

# Compute base angles
base_alpha = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_min = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_max = np.zeros([HEIGHT, WIDTH])*np.nan

lighting_color = np.zeros([HEIGHT, WIDTH])

real_alpha_min = np.zeros([HEIGHT, WIDTH])*np.nan
real_alpha_max = np.zeros([HEIGHT, WIDTH])*np.nan
real_alpha = np.zeros([HEIGHT, WIDTH])

# Blocking blocks
blocks = np.zeros([HEIGHT, WIDTH])

# [0. ; 1.] ratio = entering light angle / base light angle
# are we receiving all the lighting that we can accept without blocking element?
real_ratio = np.zeros([HEIGHT, WIDTH])

computeBaseSector()

print("Remise en état : OK")

Remise en état : OK


Et maintenant qu'on a dit ça, que peut on faire ?


### Solution 1 : le ratio comme mesure de visibilité de la cible (cellule complète ou mur)

**TODO: remplacer "éclairage" par "vision" ou quelque chose approchant**

- le calcul de vision d'un mur devrait tenir compte de la nature de ses voisins qui contribuent à son éclairage (gauche et bas)
- l'algorithme devient :
    - si le voisin de gauche est un mur, considérer uniquement l'entrée de la lumière via le segment [(x, y); (x+1,y)]
    - si le voisin du dessous est un mur, considérer uniquement l'entrée de la lumière via le segment [(x, y); (x,y+1)]
    - oui, oui, il reste le cas spécial de l'angle... on y reviendra après
- ça veut dire que la présence de blocs doit être pris en compte très tôt au moment des calculs liés aux secteurs de base :
    - soit on le prend directement en compte dans base_* mais il est difficile de faire des précalculs génériques puisque ceux ci deviennent dépendant de la position des obstacles
    - soit on stocke des calculs supplémentaires dans d'autres tableaux dans lesquels on viendra piocher au besoin
    

In [30]:
# Compute base angles (vertical walls)
base_alpha_v = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_min_v = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_max_v = np.zeros([HEIGHT, WIDTH])*np.nan

# Compute base angles (horizontal walls)
base_alpha_h = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_min_h = np.zeros([HEIGHT, WIDTH])*np.nan
base_alpha_max_h = np.zeros([HEIGHT, WIDTH])*np.nan

def getAlphaBeta(a, b):
    alpha = np.degrees(np.math.atan2(np.linalg.det([a,b]),np.dot(a,b)))
    beta = np.degrees(np.math.atan2(np.linalg.det([vx, a]), np.dot(vx, a)))
    return(alpha, beta)

def setBaseAlphaH(x, y, a, b):
    (alpha, beta) = getAlphaBeta(a, b)
#    print(f"({x}, {y}) alpha:{alpha}, beta:{beta}")
    base_alpha_h[y, x] = alpha
    base_alpha_min_h[y, x] = beta
    base_alpha_max_h[y, x] = beta + alpha

def setBaseAlphaV(x, y, a, b):
    (alpha, beta) = getAlphaBeta(a, b)
#    print(f"({x}, {y}) alpha:{alpha}, beta:{beta}")
    base_alpha_v[y, x] = alpha
    base_alpha_min_v[y, x] = beta
    base_alpha_max_v[y, x] = beta + alpha

def computeBaseSector_Walls():
    # First line
    y = 0
    for x in range(1,WIDTH):
        a = np.array([x, 0]) - L
        b = np.array([x, 1]) - L
        setBaseAlphaH(x, y, a, a)
        setBaseAlphaV(x, y, a, b)
    
    # First Column
    x = 0
    for y in range(1, HEIGHT):
        a = np.array([1, y]) - L
        b = np.array([0, y]) - L
        setBaseAlphaH(x, y, a, b)
        setBaseAlphaV(x, y, a, a)

    # All the other cells
    for y in range(1, HEIGHT):
        for x in range(1, WIDTH):
            a = np.array([x+1, y]) - L
            b = np.array([x, y]) - L
            setBaseAlphaH(x, y, a, b)
            a = np.array([x, y]) - L
            b = np.array([x, y+1]) - L
            setBaseAlphaV(x, y, a, b)
            

computeBaseSector_Walls()
print("Base sectors: OK")
print(f"(1, 1) alpha:{base_alpha[1, 1]} alphaH:{base_alpha_h[1, 1]} alphaV:{base_alpha_v[1,1]}")

Base sectors: OK
(1, 1) alpha:53.13010235415598 alphaH:26.565051177077994 alphaV:26.56505117707799


In [31]:
def setRealAlpha2(x, y, a_min, a_max, blocktype=""):
    real_alpha_min[y, x] = a_min
    real_alpha_max[y, x] = a_max
    real_alpha[y, x] = a_max - a_min
    # Watch out for the 1.0 -> 0.9999... 1.0 effect
    # --- PATCH for walls ---
    if (blocktype=="v"):
        real_ratio[y, x] = round(real_alpha[y, x] / base_alpha_v[y, x], ROUNDING_POSITION)
    elif (blocktype=="h"):
        real_ratio[y, x] = round(real_alpha[y, x] / base_alpha_h[y, x], ROUNDING_POSITION)
    else:
        real_ratio[y, x] = round(real_alpha[y, x] / base_alpha[y, x], ROUNDING_POSITION)
    # if we block lighting keep only the ratio and reset all other elements
    # with the ratio we know if the blocking cell receive light (obstacle must be displayed)
    if (blocks[y, x] == 1):
        real_alpha[y, x] = 0.
        real_alpha_min[y, x] = 0.
        real_alpha_max[y, x] = 0.
    return(real_alpha_min[y, x], real_alpha_max[y, x])

# For each cell, compute the incident light
def computeSector2(x, y):
    # Already done
    if (not np.isnan(real_alpha_min[y, x]) and not np.isnan(real_alpha_max[y, x])):
        return (real_alpha_min[y, x], real_alpha_max[y, x])
    # Origin
    if (x == 0 and y == 0):
        return (real_alpha_min[0, 0], real_alpha_max[0, 0])
    # First row (only x-1 is needed)
    if (y == 0):
        prevx = computeSector(x-1, y)
        real = intersect(prevx[0], prevx[1], base_alpha_min[y, x], base_alpha_max[y, x])
        return setRealAlpha(x, y, real[0], real[1])
    # First column (only y-1 is needed)
    if (x == 0):
        prevy = computeSector(x, y-1)
        real = intersect(prevy[0], prevy[1], base_alpha_min[y, x], base_alpha_max[y, x])
        return setRealAlpha(x, y, real[0], real[1])
    # The other cells (x-1 AND y-1 control the flow of light entering this cell)
    else:
        prevx = computeSector2(x-1, y)
        prevy = computeSector2(x, y-1)
        # --- PATCH for walls ---
        btype = ""
        if (blocks[y, x]==1 and blocks[y, x-1]==1):
            btype="h"
            real = intersect(prevy[0], prevy[1], base_alpha_min_h[y, x], base_alpha_max_h[y, x])
            print(f"({x},{y}) blocks H : real({real[0], real[1]})")
        elif (blocks[y, x]==1 and blocks[y-1, x]==1):
            btype="v"
            real = intersect(prevx[0], prevx[1], base_alpha_min_v[y, x], base_alpha_max_v[y, x])
            print(f"({x},{y}) blocks V : real({real[0], real[1]})")
        else:    
            real = intersect(prevx[0], prevx[1], base_alpha_min[y, x], base_alpha_max[y, x])
            real2 = intersect(prevy[0], prevy[1], base_alpha_min[y, x], base_alpha_max[y, x])
            real = union(real[0], real[1], real2[0], real2[1])
        return setRealAlpha2(x, y, real[0], real[1], btype)

resetArrays()

blocks[1, 2] = 1
blocks[2, 2] = 1

computeSector2(WIDTH-1, HEIGHT-1)
convertLightingRatioToColor()

print(f"Light source angles ({light_angles[0]}, {light_angles[1]})\n")
dumpdata("Real Ratio", real_ratio)    
fakeDisplay()

(2,2) blocks V : real((45.0, 59.03624346792648))
Light source angles (-180.0, 180.0)

## Real Ratio ##
      0    1      2      3      4
4  1.0  1.0  0.733  0.120  0.000
3  1.0  1.0  0.356  0.000  0.000
2  1.0  1.0  1.000  0.000  0.000
1  1.0  1.0  1.000  0.139  0.295
0  1.0  1.0  1.000  1.000  1.000



,0,1,2,3,4
4,200,200,120,40,0
3,200,200,80,0,0
2,200,200,200,0,0
1,200,200,200,40,80
0,200,200,200,200,200


In [32]:
resetArrays()

blocks[2, 1] = 1
blocks[2, 2] = 1

computeSector2(WIDTH-1, HEIGHT-1)
convertLightingRatioToColor()

print(f"Light source angles ({light_angles[0]}, {light_angles[1]})\n")
dumpdata("Real Ratio", real_ratio)    
fakeDisplay()

(2,2) blocks H : real((30.96375653207353, 45.00000000000001))
Light source angles (-180.0, 180.0)

## Real Ratio ##
      0      1    2      3      4
4  1.0  0.295  0.0  0.000  0.000
3  1.0  0.139  0.0  0.000  0.120
2  1.0  1.000  1.0  0.356  0.733
1  1.0  1.000  1.0  1.000  1.000
0  1.0  1.000  1.0  1.000  1.000



,0,1,2,3,4
4,200,80,0,0,0
3,200,40,0,0,40
2,200,200,200,80,120
1,200,200,200,200,200
0,200,200,200,200,200


### Cas des angles

In [33]:
resetArrays()

blocks[2, 1] = 1
blocks[1, 2] = 1
blocks[2, 2] = 1

computeSector2(WIDTH-1, HEIGHT-1)
convertLightingRatioToColor()

print(f"Light source angles ({light_angles[0]}, {light_angles[1]})\n")
dumpdata("Real Ratio", real_ratio)    
fakeDisplay()

(2,2) blocks H : real((0.0, 0.0))
Light source angles (-180.0, 180.0)

## Real Ratio ##
      0      1    2      3      4
4  1.0  0.295  0.0  0.000  0.000
3  1.0  0.139  0.0  0.000  0.000
2  1.0  1.000  0.0  0.000  0.000
1  1.0  1.000  1.0  0.139  0.295
0  1.0  1.000  1.0  1.000  1.000



,0,1,2,3,4
4,200,80,0,0,0
3,200,40,0,0,0
2,200,200,0,0,0
1,200,200,200,40,80
0,200,200,200,200,200


Là il va sérieusement falloir tricher parce qu'aucun rayon lumineux ne parvient à C(2,2) et c'est normal... sauf qu'on voudrait bien que cette cellule soit éclairée

#### Solution 1
- si le voisin de gauche est un mur ET si le voisin du dessous est un mur
    - le ratio est le moyenne du ratio des voisins gauche et dessous

In [34]:
def setRealAlpha3(x, y, a_min, a_max, blocktype=""):
    real_alpha_min[y, x] = a_min
    real_alpha_max[y, x] = a_max
    real_alpha[y, x] = a_max - a_min
    # Watch out for the 1.0 -> 0.9999... 1.0 effect
    if (blocktype=="v"):
        real_ratio[y, x] = round(real_alpha[y, x] / base_alpha_v[y, x], ROUNDING_POSITION)
    elif (blocktype=="h"):
        real_ratio[y, x] = round(real_alpha[y, x] / base_alpha_h[y, x], ROUNDING_POSITION)
    elif (blocktype=="a"):
        real_ratio[y, x] = round((real_ratio[y-1, x] + real_ratio[y, x-1])/2., ROUNDING_POSITION)
    else:
        real_ratio[y, x] = round(real_alpha[y, x] / base_alpha[y, x], ROUNDING_POSITION)
    # with the ratio we know if the blocking cell receive light (obstacle must be displayed)
    if (blocks[y, x] == 1):
        real_alpha[y, x] = 0.
        real_alpha_min[y, x] = 0.
        real_alpha_max[y, x] = 0.
    return(real_alpha_min[y, x], real_alpha_max[y, x])

def computeSector3(x, y):
    # Already done
    if (not np.isnan(real_alpha_min[y, x]) and not np.isnan(real_alpha_max[y, x])):
        return (real_alpha_min[y, x], real_alpha_max[y, x])
    # Origin
    if (x == 0 and y == 0):
        return (real_alpha_min[0, 0], real_alpha_max[0, 0])
    # First row (only x-1 is needed)
    if (y == 0):
        prevx = computeSector(x-1, y)
        real = intersect(prevx[0], prevx[1], base_alpha_min[y, x], base_alpha_max[y, x])
        return setRealAlpha(x, y, real[0], real[1])
    # First column (only y-1 is needed)
    if (x == 0):
        prevy = computeSector(x, y-1)
        real = intersect(prevy[0], prevy[1], base_alpha_min[y, x], base_alpha_max[y, x])
        return setRealAlpha(x, y, real[0], real[1])
    # The other cells (x-1 AND y-1 control the flow of light entering this cell)
    else:
        prevx = computeSector3(x-1, y)
        prevy = computeSector3(x, y-1)
        real = []
        btype = ""
        if (blocks[y, x]==1 and blocks[y, x-1]==1):
            btype="h"
            real = intersect(prevy[0], prevy[1], base_alpha_min_h[y, x], base_alpha_max_h[y, x])
            print(f"({x},{y}) blocks H : real({real[0], real[1]})")
        elif (blocks[y, x]==1 and blocks[y-1, x]==1):
            btype="v"
            real = intersect(prevx[0], prevx[1], base_alpha_min_v[y, x], base_alpha_max_v[y, x])
            print(f"({x},{y}) blocks V : real({real[0], real[1]})")
        else:    
            real = intersect(prevx[0], prevx[1], base_alpha_min[y, x], base_alpha_max[y, x])
            real2 = intersect(prevy[0], prevy[1], base_alpha_min[y, x], base_alpha_max[y, x])
            real = union(real[0], real[1], real2[0], real2[1])
        if (btype!="") :
            if (blocks[y-1, x] == blocks[y, x-1] == 1):
                print(f"({x},{y}) angle")
                btype="a"
        return setRealAlpha3(x, y, real[0], real[1], btype)

resetArrays()

blocks[2, 1] = 1
blocks[1, 2] = 1
blocks[2, 2] = 1

#blocks[1, 2] = 1


computeSector3(WIDTH-1, HEIGHT-1)
convertLightingRatioToColor()

print(f"Light source angles ({light_angles[0]}, {light_angles[1]})\n")
dumpdata("Real Ratio", real_ratio)    
fakeDisplay()

(2,2) blocks H : real((0.0, 0.0))
(2,2) angle
Light source angles (-180.0, 180.0)

## Real Ratio ##
      0      1    2      3      4
4  1.0  0.295  0.0  0.000  0.000
3  1.0  0.139  0.0  0.000  0.000
2  1.0  1.000  1.0  0.000  0.000
1  1.0  1.000  1.0  0.139  0.295
0  1.0  1.000  1.0  1.000  1.000



,0,1,2,3,4
4,200,80,0,0,0
3,200,40,0,0,0
2,200,200,200,0,0
1,200,200,200,40,80
0,200,200,200,200,200


#### Quelques cas...

In [35]:
resetArrays()

blocks[3, 2] = 1
blocks[2, 3] = 1
blocks[3, 3] = 1

blocks[1, 2] = 1

computeSector3(WIDTH-1, HEIGHT-1)
convertLightingRatioToColor()

print(f"Light source angles ({light_angles[0]}, {light_angles[1]})\n")
dumpdata("Real Ratio", real_ratio)    
fakeDisplay()

(3,3) blocks H : real((0.0, 0.0))
(3,3) angle
Light source angles (-180.0, 180.0)

## Real Ratio ##
      0    1      2      3      4
4  1.0  1.0  0.279  0.000  0.000
3  1.0  1.0  1.000  0.500  0.000
2  1.0  1.0  0.500  0.000  0.000
1  1.0  1.0  1.000  0.139  0.295
0  1.0  1.0  1.000  1.000  1.000



,0,1,2,3,4
4,200,200,80,0,0
3,200,200,200,120,0
2,200,200,120,0,0
1,200,200,200,40,80
0,200,200,200,200,200


In [36]:
resetArrays()

blocks[3, 3] = 1
blocks[2, 4] = 1
blocks[3, 4] = 1

blocks[1, 0] = 1

computeSector3(WIDTH-1, HEIGHT-1)
convertLightingRatioToColor()

print(f"Light source angles ({light_angles[0]}, {light_angles[1]})\n")
dumpdata("Real Ratio", real_ratio)    
fakeDisplay()

(4,3) blocks H : real((0.0, 0.0))
(4,3) angle
Light source angles (-180.0, 180.0)

## Real Ratio ##
      0    1    2    3     4
4  0.0  0.0  0.0  0.0  0.00
3  0.0  0.0  0.0  0.5  0.75
2  0.0  0.0  0.5  1.0  1.00
1  1.0  0.5  1.0  1.0  1.00
0  1.0  1.0  1.0  1.0  1.00



,0,1,2,3,4
4,0,0,0,0,0
3,0,0,0,120,160
2,0,0,120,200,200
1,200,120,200,200,200
0,200,200,200,200,200
